# DB Queries

### Query a

In [ ]:
query="""
MATCH (m:Customer)-[:USE]->(t:Terminal)<-[:USE]-(n:Customer)
WHERE m <> n
WITH m, n, COUNT(DISTINCT t) AS sharedTerminals
WHERE sharedTerminals >= 4

MATCH (m)-[:PERFORM]->(tm:Transaction)
MATCH (n)-[:PERFORM]->(tn:Transaction)
WITH
    m, n, sharedTerminals,
    COUNT(DISTINCT tm) AS m_tx_count,
    COUNT(DISTINCT tn) AS n_tx_count

WHERE abs(m_tx_count - n_tx_count) <= 2

RETURN
    m.id AS customer_M,
    n.id AS customer_N,
    sharedTerminals,
    m_tx_count,
    n_tx_count
ORDER BY sharedTerminals DESC;
"""

### Query b